In [1]:
import os
import sys

In [2]:
# Theodore's paths (do not run this cell)
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
#os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.1,org.apache.spark:spark-streaming-kafka-0-8_2.11:2.2.0 --jars spark-sql-kafka-0-10_2.11-2.4.1.jar pyspark-shell'
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.4 pyspark-shell'

In [2]:

os.environ["SPARK_HOME"]="/home/user1/trainig_material/spark-2.4.4-bin-hadoop2.7"
#os.environ["SPARK_HOME"]='/usr/local/bin/spark-2.4.4-bin-hadoop2.6'
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
#os.environ["HIVE_HOME"] ="/usr/lib/hive"
#os.environ["PATH"] =os.environ["HIVE_HOME"]+"/bin"
#os.environ["JAVA_HOME"]="/usr/java/jdk1.7.0_67-cloudera"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
#os.environ["PYSPARK_PYTHON"] = "/home/cloudera/anaconda3/bin/python3.7" 
#os.environ["PYSPARK_DRIVER_PYTHON"] = "/home/cloudera/anaconda3/bin/python3.7"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")


os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.4 pyspark-shell'

In [3]:

!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 2.4.4
      /_/
                        
Using Scala version 2.11.12, OpenJDK 64-Bit Server VM, 1.8.0_292
Branch 
Compiled by user  on 2019-08-27T21:21:38Z
Revision 
Url 
Type --help for more information.


In [4]:
# Spark
from pyspark import SparkContext, SparkConf
# Spark Streaming
from pyspark.streaming import StreamingContext
# Kafka
#from pyspark.streaming.kafka import KafkaUtils
# json parsing
from pyspark.sql import Row,SQLContext
from pyspark import SparkConf,SparkContext
from pyspark.sql import SQLContext,HiveContext

In [5]:
sc = SparkContext("local", "twitter")
sc.setLogLevel("WARN")

In [5]:
sc.stop()

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [7]:
spark.sql("show databases").show()

+----------------+
|    databaseName|
+----------------+
|         default|
|            test|
|twitter_analysis|
+----------------+



In [8]:
import pandas as pd
from datetime import datetime, timedelta
import json

import tweepy
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from textblob import TextBlob

from kafka import KafkaClient
from kafka import KafkaProducer
from kafka import KafkaConsumer
from kafka import TopicPartition
from kafka.errors import KafkaError

In [88]:
consumer = ''

In [89]:
# not utilizing a structured streaming environment, this will allow us to save tweets in consumer 
#nd not neet to run producer and streaming simultaneosly avoiding error:420
consumer = KafkaConsumer('final_twitter', 
                         group_id='1', # Using group so that it doesn't re-read messages
                         auto_offset_reset='earliest', 
                         enable_auto_commit=True, 
                         bootstrap_servers=['localhost:9092'], 
                         value_deserializer=lambda m: json.loads(m))
# Create TopicPartition object so we can keep track of offset and know when to exit consumer loop later
tp = TopicPartition('final_twitter',0)

In [90]:
rows = [] # Put in separate cell so we don't clear rows every time we run next cell

In [91]:
date_format = '%a %b %d %H:%M:%S %Y' # To convert created_at to Datetime object

#list of dicts, this will save tweets from consumer
for msg in consumer:
    # Obtain the last offset value, so that we know when to exit loop
    current_position = consumer.position(tp) # Store current offset pos so we can seek to it after
    consumer.seek_to_end(tp)
    last_offset = consumer.position(tp)
    
    consumer.seek(tp, current_position) # Seek back to where we were
    #print("\nCurrent offset:", msg.offset)
    #print("Last offset:", last_offset)
    
    # Comment this out to continue even if consumer catches up to producer
    if msg.offset >= last_offset - 1: 
        break
    
    # this converts consumer to type dict
    row = msg.value
    
    # pull the hashtags
    if len(row['entities']['hashtags']) > 0:
        #hashtag_text = [row['entities']['hashtags'][0]['text'] # Get text string of first hashtag
        hashtag_text = [row['entities']['hashtags'][i]['text'].lower() for i, tag in enumerate(row['entities']['hashtags'])]
    else:
        #continue # Commented this out since if we continue, the next line wouldn't execute and we wouldn't store the tweet
        hashtag_text = []
    # create a row and add it to the list, we only need Tweet Id, User Id, Text, hashtags, and event time
    row = Row(TweetID=row['id'], 
              Text=row['text'], 
              UserID=row['user']['id'], 
              Hashtags=hashtag_text,
              #Event_time=row['created_at']
              Event_time=datetime.strptime(row['created_at'].replace('+0000 ', ''), date_format)
    )
    
    rows.append(row)
    #print(row)

In [92]:
# sanity check
print(rows[:10]) # First 10 rows
print('\nNumber of tweets collected: {}'.format(len(rows)))

[Row(Event_time=datetime.datetime(2021, 7, 28, 15, 55, 49), Hashtags=[], Text='Speechless.', TweetID=1420412688723693571, UserID=906472343857328128), Row(Event_time=datetime.datetime(2021, 7, 28, 15, 55, 49), Hashtags=[], Text='RT @ijaicool: Lee Zii Jia at the Olympics: https://t.co/zFL96N028m', TweetID=1420412689319223297, UserID=868734692232581120), Row(Event_time=datetime.datetime(2021, 7, 28, 15, 55, 49), Hashtags=[], Text='I wish I was this wise and strong when I was her age', TweetID=1420412689747226631, UserID=378120117), Row(Event_time=datetime.datetime(2021, 7, 28, 15, 55, 49), Hashtags=[], Text='KE olympics au fic when?', TweetID=1420412689818562561, UserID=1229910244203139072), Row(Event_time=datetime.datetime(2021, 7, 28, 15, 55, 50), Hashtags=[], Text='Olympics?\nMore like Olamepics. Amirite?', TweetID=1420412690380382210, UserID=64345412), Row(Event_time=datetime.datetime(2021, 7, 28, 15, 55, 50), Hashtags=[], Text="RT @BrianBertie1998: It's over! Kento Momota, the world 

In [93]:
# convert the list of dicts to a spark dataframe as per the requirements
df = spark.createDataFrame(rows)

In [94]:
df.show()

+-------------------+----------+-------------------------+-------------------+-------------------+
|         Event_time|  Hashtags|                     Text|            TweetID|             UserID|
+-------------------+----------+-------------------------+-------------------+-------------------+
|2021-07-28 15:55:49|        []|              Speechless.|1420412688723693571| 906472343857328128|
|2021-07-28 15:55:49|        []|     RT @ijaicool: Lee...|1420412689319223297| 868734692232581120|
|2021-07-28 15:55:49|        []|     I wish I was this...|1420412689747226631|          378120117|
|2021-07-28 15:55:49|        []|     KE olympics au fi...|1420412689818562561|1229910244203139072|
|2021-07-28 15:55:50|        []|     Olympics?
More li...|1420412690380382210|           64345412|
|2021-07-28 15:55:50|        []|     RT @BrianBertie19...|1420412691118624768|1311798800231489538|
|2021-07-28 15:55:50|[olympics]|     RT @binance: Woul...|1420412691890507777|1346472473240354817|
|2021-07-2

## Number of tweets in last 30 second window

In [95]:
df.createOrReplaceTempView("tweet_df")

In [96]:
# Probably won't capture anything unless run right away, since it uses current time instead of most recent tweet
# Maybe better to use below method
spark.sql("""
    SELECT count(*) as num_tweets_last_30
    FROM tweet_df
    WHERE Event_time between (current_timestamp() - INTERVAL 30 SECOND ) 
                                AND current_timestamp()
""").show()

+------------------+
|num_tweets_last_30|
+------------------+
|                 0|
+------------------+



In [97]:
# Get the most recent tweet event_time captured, store as datetime
# We can use this for all other queries to compare against most recent tweet (instead of current time, since it might be delayed)
most_recent_tweet = spark.sql("""
    SELECT Event_time
    FROM tweet_df
    ORDER BY Event_time 
    DESC LIMIT 1""").collect()[0].Event_time

In [98]:
print('Datetime of most recently captured tweet is: {}'.format(most_recent_tweet)) 

Datetime of most recently captured tweet is: 2021-07-28 17:54:56


In [99]:

query = """
    SELECT COUNT(*) as Num_Tweets_Last_30_From_Newest_Tweet
    FROM tweet_df
    WHERE Event_time between ('{}' - INTERVAL 30 SECOND ) 
                                AND '{}'""".format(most_recent_tweet, most_recent_tweet)
print(query)
num_tweets_last_30 = spark.sql(query)
num_tweets_last_30.show()


    SELECT COUNT(*) as Num_Tweets_Last_30_From_Newest_Tweet
    FROM tweet_df
    WHERE Event_time between ('2021-07-28 17:54:56' - INTERVAL 30 SECOND ) 
                                AND '2021-07-28 17:54:56'
+------------------------------------+
|Num_Tweets_Last_30_From_Newest_Tweet|
+------------------------------------+
|                                 231|
+------------------------------------+



## Num active users last 30 seconds

In [39]:
# Actual last 30 seconds from current time
query = """
    SELECT count(distinct(UserID)) as Num_Active_Users_Last_30_From_Current
    FROM tweet_df
    WHERE Event_time between (current_timestamp() - INTERVAL 30 SECOND ) 
                                AND current_timestamp()""".format(most_recent_tweet)
spark.sql(query).show()


+-------------------------------------+
|Num_Active_Users_Last_30_From_Current|
+-------------------------------------+
|                                    0|
+-------------------------------------+



In [40]:
# 30 seconds prior to most recently received tweet
query = """
    SELECT count(distinct(UserID)) as Num_Active_Users_Last_30_From_Newest_Tweet
    FROM tweet_df
    WHERE Event_time between ('{}' - INTERVAL 30 SECOND ) 
                                AND '{}'""".format(most_recent_tweet, most_recent_tweet)
print(query)
spark.sql(query).show()


    SELECT count(distinct(UserID)) as Num_Active_Users_Last_30_From_Newest_Tweet
    FROM tweet_df
    WHERE Event_time between ('2021-07-28 15:16:35' - INTERVAL 30 SECOND ) 
                                AND '2021-07-28 15:16:35'
+------------------------------------------+
|Num_Active_Users_Last_30_From_Newest_Tweet|
+------------------------------------------+
|                                       387|
+------------------------------------------+



## Most used hashtag in last 30 seconds

In [41]:
# key=lambda a: a[1] Means use the second value in the (word, num_occurrences) tuple as comparison key for max
query = """
    SELECT Hashtags
    FROM tweet_df
    WHERE Event_time between ('{}' - INTERVAL 30 SECOND ) 
                                AND '{}'""".format(most_recent_tweet, most_recent_tweet)
df_last_30 = spark.sql(query)
frequent_hashtag_tup_last_30 = max(df_last_30.select(['Hashtags']).rdd.flatMap(lambda a: a.Hashtags).countByValue().items(), key=lambda a: a[1])
print('Most frequently occuring hashtag in last 30 seconds: {}'.format(frequent_hashtag_tup_last_30[0]))
print('Num occurrences: {}'.format(frequent_hashtag_tup_last_30[1]))

Most frequently occuring hashtag in last 30 seconds: olympics
Num occurrences: 58


## Most used hashtag in complete time of analysis

In [42]:
# key=lambda a: a[1] Means use the second value in the (word, num_occurrences) tuple as comparison key for max
frequent_hashtag_tup_complete = max(df.select(['Hashtags']).rdd.flatMap(lambda a: a.Hashtags).countByValue().items(), key=lambda a: a[1])
print('Most frequently occuring hashtag in complete time of analysis: {}'.format(frequent_hashtag_tup_complete[0]))
print('Num occurrences: {}'.format(frequent_hashtag_tup_complete[1]))

Most frequently occuring hashtag in complete time of analysis: olympics
Num occurrences: 94


## Moving average of a given Hashtag every 1 minute

In [43]:
from pyspark.sql.window import Window
from pyspark.sql import functions as func
from pyspark.sql.functions import explode
# I basically followed this tutorial: 
# https://www.linkedin.com/pulse/time-series-moving-average-apache-pyspark-laurent-weichberger/


In [44]:
# Create a "seven day" Window from seven days previous to the current day (row), using previous casting of timestamp to long (number of seconds).

# remember a start value of "-1" means one off before the current row, and we are taking the timestamp as a long and comparing it to the rangeBetween amount of time.

# Remember an end value of 0 is the current row.
# We use 6 days because rangeBetween is inclusive of start & end values.
# Thank you Dr. Kevin Maguire for the bug fix!
#windowSpec = Window.orderBy(func.col("Event_time").cast('long')).rangeBetween(2, 0) # Using 3 seconds for now. Set to 59 for 1 minute
windowSpec = Window.orderBy(func.col("Event_time").cast('long')).rangeBetween(2, 0) # Using 3 seconds for now. Set to 59 for 1 minute

In [45]:
# Explode the dataframe so that each element in each Hashtag array has its own row, 
# and so that the Hashtags column now only contains one hashtag string
df_exploded = df.withColumn('Hashtags', explode('Hashtags'))
df_exploded.show()

+-------------------+----------------------+--------------------+-------------------+-------------------+
|         Event_time|              Hashtags|                Text|            TweetID|             UserID|
+-------------------+----------------------+--------------------+-------------------+-------------------+
|2021-07-28 14:52:41|        winitwednesday|RT @HelleniaUK: #...|1420396798322040832|          348526826|
|2021-07-28 14:52:41|           competition|RT @HelleniaUK: #...|1420396798322040832|          348526826|
|2021-07-28 14:52:41|      blacklivesmatter|RT @StanisElsborg...|1420396799014150147|         2438109488|
|2021-07-28 14:52:41|          raisearacket|RT @bwfmedia: The...|1420396799454588930|1390696881202688000|
|2021-07-28 14:52:41|             tokyo2020|RT @bwfmedia: The...|1420396799454588930|1390696881202688000|
|2021-07-28 14:52:41|              olympics|RT @bwfmedia: The...|1420396799454588930|1390696881202688000|
|2021-07-28 14:52:41|          shameonjapan|RT

In [ ]:
# Now create a new column which has the count of 

In [46]:
# Filter DF such that it only contains rows with our hashtag of interest
df2 = df_exploded.filter(df_exploded.Hashtags == frequent_hashtag_tup_complete[0])

In [47]:
df2.show()

+-------------------+--------+--------------------+-------------------+-------------------+
|         Event_time|Hashtags|                Text|            TweetID|             UserID|
+-------------------+--------+--------------------+-------------------+-------------------+
|2021-07-28 14:52:41|olympics|RT @bwfmedia: The...|1420396799454588930|1390696881202688000|
|2021-07-28 14:52:41|olympics|RT @DIN_NAYU: #sh...|1420396799655747591|1245984503098564608|
|2021-07-28 14:52:42|olympics|RT @ITFTennis: On...|1420396802512089088|          562036192|
|2021-07-28 14:52:42|olympics|Arabian iptv list...|1420396803841830915| 838513081529565184|
|2021-07-28 14:52:42|olympics|RT @FeminaIndia: ...|1420396805402071040| 962504002641555457|
|2021-07-28 14:52:42|olympics|RT @JioJio7907972...|1420396805976690694|1147786434901712896|
|2021-07-28 14:52:42|olympics|RT @UN_Women: We ...|1420396806014390276|1055987824749625344|
|2021-07-28 14:52:43|olympics|RT @2652450828Hi:...|1420396808312803331|128145659

In [48]:
from pyspark.sql.functions import lit
df2 = df2.withColumn('count', lit(1.0)) # Simply add a new column and set each val to 1 (used for moving avg later)
df2.show()

+-------------------+--------+--------------------+-------------------+-------------------+-----+
|         Event_time|Hashtags|                Text|            TweetID|             UserID|count|
+-------------------+--------+--------------------+-------------------+-------------------+-----+
|2021-07-28 14:52:41|olympics|RT @bwfmedia: The...|1420396799454588930|1390696881202688000|  1.0|
|2021-07-28 14:52:41|olympics|RT @DIN_NAYU: #sh...|1420396799655747591|1245984503098564608|  1.0|
|2021-07-28 14:52:42|olympics|RT @ITFTennis: On...|1420396802512089088|          562036192|  1.0|
|2021-07-28 14:52:42|olympics|Arabian iptv list...|1420396803841830915| 838513081529565184|  1.0|
|2021-07-28 14:52:42|olympics|RT @FeminaIndia: ...|1420396805402071040| 962504002641555457|  1.0|
|2021-07-28 14:52:42|olympics|RT @JioJio7907972...|1420396805976690694|1147786434901712896|  1.0|
|2021-07-28 14:52:42|olympics|RT @UN_Women: We ...|1420396806014390276|1055987824749625344|  1.0|
|2021-07-28 14:52:43

In [49]:
# Edit: Not sure if we need this
# We do a wordcount on Event_time, because at this point each Event_time row is one occurance for the most frequent hashtag
from pyspark.sql.types import DoubleType
import pyspark.sql.functions as f
df3 = df2 \
        .groupBy('Event_time') \
        .count() \
        .sort('Event_time', ascending=True)
df3.show()

+-------------------+-----+
|         Event_time|count|
+-------------------+-----+
|2021-07-28 14:52:41|    2|
|2021-07-28 14:52:42|    5|
|2021-07-28 14:52:43|    1|
|2021-07-28 15:15:53|    4|
|2021-07-28 15:15:54|    2|
|2021-07-28 15:15:55|    2|
|2021-07-28 15:15:56|    3|
|2021-07-28 15:15:57|    1|
|2021-07-28 15:15:58|    3|
|2021-07-28 15:15:59|    1|
|2021-07-28 15:16:00|    1|
|2021-07-28 15:16:01|    1|
|2021-07-28 15:16:02|    3|
|2021-07-28 15:16:03|    1|
|2021-07-28 15:16:04|    6|
|2021-07-28 15:16:05|    2|
|2021-07-28 15:16:06|    3|
|2021-07-28 15:16:07|    3|
|2021-07-28 15:16:08|    1|
|2021-07-28 15:16:09|    2|
+-------------------+-----+
only showing top 20 rows



In [50]:
# SparkSQL method - not sure if this is correct
df3.createOrReplaceTempView("df3")
spark.sql(
    """SELECT *, mean(count) OVER (
        ORDER BY CAST(Event_time AS timestamp) 
        RANGE BETWEEN INTERVAL 60 SECONDS PRECEDING AND CURRENT ROW
     ) AS Moving_Average FROM df3""").show()

+-------------------+-----+------------------+
|         Event_time|count|    Moving_Average|
+-------------------+-----+------------------+
|2021-07-28 14:52:41|    2|               2.0|
|2021-07-28 14:52:42|    5|               3.5|
|2021-07-28 14:52:43|    1|2.6666666666666665|
|2021-07-28 15:15:53|    4|               4.0|
|2021-07-28 15:15:54|    2|               3.0|
|2021-07-28 15:15:55|    2|2.6666666666666665|
|2021-07-28 15:15:56|    3|              2.75|
|2021-07-28 15:15:57|    1|               2.4|
|2021-07-28 15:15:58|    3|               2.5|
|2021-07-28 15:15:59|    1|2.2857142857142856|
|2021-07-28 15:16:00|    1|             2.125|
|2021-07-28 15:16:01|    1|               2.0|
|2021-07-28 15:16:02|    3|               2.1|
|2021-07-28 15:16:03|    1|               2.0|
|2021-07-28 15:16:04|    6|2.3333333333333335|
|2021-07-28 15:16:05|    2|2.3076923076923075|
|2021-07-28 15:16:06|    3| 2.357142857142857|
|2021-07-28 15:16:07|    3|               2.4|
|2021-07-28 1

## Save results from 7a to Hive

In [100]:
# Add timestamp to this record in case we want to make sense of the data in the hive table later
# Also, reorder columns so timestamp is on left
table_to_write = num_tweets_last_30.withColumn('End_of_Capturing_Window', lit(most_recent_tweet))
table_to_write = table_to_write.select('End_of_Capturing_Window', 'Num_Tweets_Last_30_From_Newest_Tweet')
table_to_write.show()

+-----------------------+------------------------------------+
|End_of_Capturing_Window|Num_Tweets_Last_30_From_Newest_Tweet|
+-----------------------+------------------------------------+
|    2021-07-28 17:54:56|                                 231|
+-----------------------+------------------------------------+



In [101]:
# Create db and table
spark.sql("""
    CREATE DATABASE IF NOT EXISTS twitter_analysis
""")

spark.sql("""
    USE twitter_analysis
""").show()

spark.sql("""
    CREATE TABLE IF NOT EXISTS tweets_last_30(
        End_of_Capturing_Window String, 
        Num_Tweets_Last_30_From_Newest_Tweet String
    )
""").show()

++
||
++
++

++
||
++
++



In [102]:
table_to_write.write.mode("append").format('hive').saveAsTable("twitter_analysis.tweets_last_30")

In [103]:
spark.sql("""
    SELECT *
    FROM tweets_last_30
    ORDER BY End_of_Capturing_Window
""").show()

+-----------------------+------------------------------------+
|End_of_Capturing_Window|Num_Tweets_Last_30_From_Newest_Tweet|
+-----------------------+------------------------------------+
|    2021-07-25 18:42:01|                                  84|
|    2021-07-28 14:52:44|                                 178|
|    2021-07-28 15:16:35|                                 412|
|    2021-07-28 15:35:56|                                 328|
|    2021-07-28 15:55:57|                                 323|
|    2021-07-28 17:54:56|                                 231|
+-----------------------+------------------------------------+

